In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import numpy as np
import os
import time
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
DATASET_PATH='dataset\\preprocesd_cicids.csv'
print(DATASET_PATH)

dataset\preprocesd_cicids.csv


In [4]:
# ,nrows=10000, header=0
start = time.time()
df=pd.read_csv(DATASET_PATH)
df.head()
print("Time taken to load the data: ", time.time()-start," seconds")

Time taken to load the data:  22.986497402191162  seconds


In [5]:
df.shape

(2520798, 79)

In [6]:
def correlation(dataset, threshold):
    col_corr = set()
    corr_matrix = dataset.corr(numeric_only=True)
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
              colname = corr_matrix.columns[i]
              col_corr.add(colname)
    return col_corr

In [7]:
corr_features = correlation(df, 0.85)
corr_features

{' Active Min',
 ' Average Packet Size',
 ' Avg Bwd Segment Size',
 ' Avg Fwd Segment Size',
 ' Bwd IAT Min',
 ' Bwd Packet Length Mean',
 ' Bwd Packet Length Std',
 ' CWE Flag Count',
 ' ECE Flag Count',
 ' Flow IAT Max',
 ' Fwd Header Length.1',
 ' Fwd IAT Max',
 ' Fwd IAT Mean',
 ' Fwd IAT Min',
 ' Fwd IAT Std',
 ' Fwd Packet Length Mean',
 ' Fwd Packet Length Std',
 ' Idle Max',
 ' Idle Min',
 ' Max Packet Length',
 ' Packet Length Mean',
 ' Packet Length Std',
 ' Packet Length Variance',
 ' SYN Flag Count',
 ' Subflow Bwd Bytes',
 ' Subflow Bwd Packets',
 ' Subflow Fwd Bytes',
 ' Total Backward Packets',
 ' Total Length of Bwd Packets',
 ' act_data_pkt_fwd',
 ' min_seg_size_forward',
 'Fwd IAT Total',
 'Fwd Packets/s',
 'Idle Mean',
 'Subflow Fwd Packets'}

In [8]:
df.drop(corr_features,axis=1,inplace=True)

In [9]:
df.shape

(2520798, 44)

In [10]:
# create a Series with the count of rows in each group
label_counts = df[' Label'].value_counts()

# create a list of labels that have less than 10,000 rows
labels_to_merge = label_counts[label_counts < 3000].index.tolist()
print(labels_to_merge)

['Bot', 'Web Attack � Brute Force', 'Web Attack � XSS', 'Infiltration', 'Web Attack � Sql Injection', 'Heartbleed']


In [11]:
# # create a new label called 'Other' and merge the labels with less than 10,000 rows
df[' Label'] = df[' Label'].apply(lambda x: 'Other' if x in labels_to_merge else x)
# group the rows by the new 'Label' column
grouped_df = df.groupby(' Label')

In [12]:
df.shape

(2520798, 44)

In [13]:
df[' Label'].value_counts()

BENIGN              2095057
DoS Hulk             172846
DDoS                 128014
PortScan              90694
DoS GoldenEye         10286
FTP-Patator            5931
DoS slowloris          5385
DoS Slowhttptest       5228
Other                  4138
SSH-Patator            3219
Name:  Label, dtype: int64

In [14]:
x = df.drop([' Label'],axis=1)
y = df[' Label']

In [15]:
# from imblearn.under_sampling import RandomUnderSampler
# rus = RandomUnderSampler(random_state=0)
# rus.fit(x, y)
# Xn, yn = rus.fit_resample(x, y)
# # Xn.value_counts()
Xn, yn = x, y

In [16]:
Xn.shape

(2520798, 43)

In [17]:
yn.shape

(2520798,)

In [18]:
yn.value_counts()

BENIGN              2095057
DoS Hulk             172846
DDoS                 128014
PortScan              90694
DoS GoldenEye         10286
FTP-Patator            5931
DoS slowloris          5385
DoS Slowhttptest       5228
Other                  4138
SSH-Patator            3219
Name:  Label, dtype: int64

In [19]:
#z-score   z = (x - mean) / std
# it can make it easier for the algorithm to learn meaningful patterns in the data
cols = list(Xn.columns)
for col in cols:
    Xn[col] = stats.zscore(Xn[col])

In [20]:
Xn.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Length of Fwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Bwd Packet Length Max,Bwd Packet Length Min,Flow Bytes/s,Flow Packets/s,...,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Init_Win_bytes_forward,Init_Win_bytes_backward,Active Mean,Active Std,Active Max,Idle Std
0,2.428596,-0.470914,-0.010425,-0.056662,-0.297774,-0.217169,-0.478315,-0.608911,0.097449,3.056580,...,NaN,NaN,NaN,NaN,-0.495454,-0.249846,-0.13337,-0.110882,-0.158458,-0.11608
1,2.438537,-0.470911,-0.011684,-0.057228,-0.297774,-0.217169,-0.475372,-0.524261,-0.048949,-0.142833,...,NaN,NaN,NaN,NaN,-0.495728,-0.221049,-0.13337,-0.110882,-0.158458,-0.11608
2,2.438590,-0.470913,-0.011684,-0.057228,-0.297774,-0.217169,-0.475372,-0.524261,-0.044407,-0.043577,...,NaN,NaN,NaN,NaN,-0.495728,-0.221049,-0.13337,-0.110882,-0.158458,-0.11608
3,1.974744,-0.470913,-0.011684,-0.057228,-0.297774,-0.217169,-0.475372,-0.524261,-0.039809,0.056908,...,NaN,NaN,NaN,NaN,-0.495591,-0.212869,-0.13337,-0.110882,-0.158458,-0.11608
4,2.428491,-0.470914,-0.010425,-0.056662,-0.297774,-0.217169,-0.478315,-0.608911,0.097449,3.056580,...,NaN,NaN,NaN,NaN,-0.495523,-0.249846,-0.13337,-0.110882,-0.158458,-0.11608


In [21]:
from sklearn.model_selection import  train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(Xn,yn,test_size=0.20,random_state=0)

In [22]:
print(np.any(np.isnan(X_train)))
print(np.all(np.isfinite(X_train)))

True
False


In [23]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Create an instance of SimpleImputer with 'mean' strategy to replace NaN values
imputer = SimpleImputer(strategy='mean')

# Fit the imputer to X_train and transform X_train and X_test with it
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Apply StandardScaler to X_train and X_test
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.svm import SVC
# model training USING SVM (suppor vector machine)
start = time.time()
svm = SVC(kernel='rbf', C=1.0, gamma='scale')
svm.fit(X_train, Y_train)
print("Time taken to train model: ", time.time()-start," seconds")

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn import metrics


# performance metrics calculation of our model over training data set
start = time.time()
Predict_X =  svm.predict(X_train)
scores = cross_val_score(svm, X_train, Y_train, cv=7)
accuracy = metrics.accuracy_score(Y_train,Predict_X)
confusion_matrix = metrics.confusion_matrix(Y_train, Predict_X)
classification = metrics.classification_report(Y_train, Predict_X)
print("Time taken to for performance matric calculation: ", time.time()-start," seconds")

In [ ]:
print()
print('--------------------------- Results --------------------------------')
print()
print ("Cross Validation Mean Score:" "\n", scores.mean())
print()
print ("Model Accuracy:" "\n", accuracy)
print()
print("Confusion matrix:" "\n", confusion_matrix)
print()
print("Classification report:" "\n", classification)
print()

In [ ]:
def plot_confusion_matrix(cm,title,cmap=None,target=None,normalize=False):

    import itertools
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('viridis')
    plt.figure(figsize=(20, 20))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target is not None:
        ticks = np.arange(len(target))
        plt.xticks(ticks, target, rotation=45)
        plt.yticks(ticks, target)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="black" if cm[i, j] > thresh else "white")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="black" if cm[i, j] > thresh else "white")
    plt.grid(False)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()
    plt.savefig(title, bbox_inches='tight', dpi=300)

In [ ]:
plot_confusion_matrix(cm=confusion_matrix ,title= 'Support Vector Machine Classifire')

In [ ]:
# performance metrics calculation of our model over test data set
start = time.time()
Predict_X =  svm.predict(X_test)
scores = cross_val_score(svm, X_test, Y_test, cv=7)
accuracy = metrics.accuracy_score(Y_test,Predict_X)
confusion_matrix = metrics.confusion_matrix(Y_test, Predict_X)
classification = metrics.classification_report(Y_test, Predict_X)
print("Time taken to for performance matric calculation: ", time.time()-start," seconds")

In [ ]:
print()
print('--------------------------- Results --------------------------------')
print()
print ("Cross Validation Mean Score:" "\n", scores.mean())
print()
print ("Model Accuracy:" "\n", accuracy)
print()
print("Confusion matrix:" "\n", confusion_matrix)
print()
print("Classification report:" "\n", classification)
print()

In [ ]:
plot_confusion_matrix(cm=confusion_matrix ,title= 'Support Vector Machine Classifire')